## Importer les bibliotheques

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time

## Extraction les liens de statistiques des clubs

In [6]:
def extract_clubs(url,season,dir):
    os.mkdir(f"{dir}/{season}")
    data=requests.get(url)
    soup=BeautifulSoup(data.text)
    table_equipes=soup.select('tbody')[0]
    tags_a=table_equipes.find_all('a')
    liens=[tag.get('href') for tag in tags_a]
    liens=[lien for lien in liens if '/squads/' in lien]
    liens=[f"https://fbref.com{lien}" for lien in liens]
    liens=[[lien.split("/")[-1],lien] for lien in liens]
    print(liens)
    header=['equipe','lien']
    f = pd.DataFrame(liens, columns=header)
    f.to_csv(f"{dir}/{season}/listeEquipes.csv")
    return f

In [7]:
res=extract_clubs("https://fbref.com/en/comps/9/2010-2011/2010-2011-Premier-League-Stats","2010-2011",".")
res

[['Manchester-United-Stats', 'https://fbref.com/en/squads/19538871/2010-2011/Manchester-United-Stats'], ['Chelsea-Stats', 'https://fbref.com/en/squads/cff3d9bb/2010-2011/Chelsea-Stats'], ['Manchester-City-Stats', 'https://fbref.com/en/squads/b8fd03ef/2010-2011/Manchester-City-Stats'], ['Arsenal-Stats', 'https://fbref.com/en/squads/18bb7c10/2010-2011/Arsenal-Stats'], ['Tottenham-Hotspur-Stats', 'https://fbref.com/en/squads/361ca564/2010-2011/Tottenham-Hotspur-Stats'], ['Liverpool-Stats', 'https://fbref.com/en/squads/822bd0ba/2010-2011/Liverpool-Stats'], ['Everton-Stats', 'https://fbref.com/en/squads/d3fd31cc/2010-2011/Everton-Stats'], ['Fulham-Stats', 'https://fbref.com/en/squads/fd962109/2010-2011/Fulham-Stats'], ['Aston-Villa-Stats', 'https://fbref.com/en/squads/8602292d/2010-2011/Aston-Villa-Stats'], ['Sunderland-Stats', 'https://fbref.com/en/squads/8ef52968/2010-2011/Sunderland-Stats'], ['West-Bromwich-Albion-Stats', 'https://fbref.com/en/squads/60c6b05f/2010-2011/West-Bromwich-Albi

,equipe,lien
0,Manchester-United-Stats,https://fbref.com/en/squads/19538871/2010-2011...
1,Chelsea-Stats,https://fbref.com/en/squads/cff3d9bb/2010-2011...
2,Manchester-City-Stats,https://fbref.com/en/squads/b8fd03ef/2010-2011...
3,Arsenal-Stats,https://fbref.com/en/squads/18bb7c10/2010-2011...
4,Tottenham-Hotspur-Stats,https://fbref.com/en/squads/361ca564/2010-2011...
5,Liverpool-Stats,https://fbref.com/en/squads/822bd0ba/2010-2011...
6,Everton-Stats,https://fbref.com/en/squads/d3fd31cc/2010-2011...
7,Fulham-Stats,https://fbref.com/en/squads/fd962109/2010-2011...
8,Aston-Villa-Stats,https://fbref.com/en/squads/8602292d/2010-2011...
9,Sunderland-Stats,https://fbref.com/en/squads/8ef52968/2010-2011...


## Extraction statistiques de chaque match 

In [8]:
def statistique_matchs(url,club,dir):
    req=requests.get(url)
    soup=BeautifulSoup(req.text)
    table_matchs=soup.select("table.stats_table")[1]
    header=[td.get_text() for td in table_matchs.select("thead tr th")]
    matches=[[element.get_text() for element in row] for row in table_matchs.select("tbody tr")]
    df = pd.DataFrame(matches, columns=header)
    liens_rapports=[]
    for ele in table_matchs.select("tbody tr"):
        liens_rapports.append("https://fbref.com"+ele.select("td")[-2].select('a')[0].get('href'))
    df['Match Report']=liens_rapports
    df=df[df["Comp"]=="Premier League"]
    df.to_csv(f"{dir}/{club}.csv")
    return df

In [ ]:
season="./2013-2014"
for index,row in res.iterrows():
    os.mkdir(f"{season}/{row['equipe']}")
    # statistique globale
    stat=statistique_matchs(row['lien'],row['equipe'],f"{season}/{row['equipe']}")
    equipe=row['equipe']
    print(equipe)

In [9]:
season="./2010-2011"
liste_ee=[]
for index,row in res.iterrows():
    os.mkdir(f"{season}/{row['equipe']}")
    # statistique globale
    try:
        stat=statistique_matchs(row['lien'],row['equipe'],f"{season}/{row['equipe']}")
    except:
        print(row['equipe'])
        liste_ee.append([row['equipe'],row['lien']])
header=['equipe','lien']
f = pd.DataFrame(liste_ee, columns=header)
f.to_csv(f"{season}/listeEquipesexcept.csv")

## Performence des joueurs

In [2]:
def performence(url,camp,dir):
    stat_=["summary","passing","passing types","defense","possession","miscellaneous stats"]
    html=pd.read_html(url)
    if camp == "Home" :
        for name,index in zip(stat_, range(3,9)):
            html[index].to_csv(f"{dir}/{name}.csv")
    else:
        for name,index in zip(stat_, range(10,16)):
            html[index].to_csv(f"{dir}/{name}.csv")


In [3]:
dir="./2018-2019/"
exception_pp=[]
equipes=os.listdir(dir)
for equipe in equipes:
    if "Stats" in equipe:
        try:
            statistiques=pd.read_csv(f"{dir}{equipe}/{equipe}.csv")
            print(equipe)
            print("-------")
            for index,round in statistiques.iterrows():
                matchweek=round['Round']
                try:
                    if "History" not in round['Match Report']:
                        performence(round['Match Report'],round['Venue'],f"{dir}{equipe}/{matchweek}")
                        print(matchweek)
                        time.sleep(2)
                except:
                    exception_pp.append([equipe,matchweek,round['Match Report']])
                    print("ERREUR : "+matchweek)
        except:
            pass
header=["team","matchweek","Match Report"]
erreur_pp = pd.DataFrame(exception_pp, columns=header)
erreur_pp.to_csv(f"{dir}/erreur_pp.csv")

Arsenal-Stats
-------
Matchweek 1
Matchweek 2
Matchweek 3
Matchweek 4
Matchweek 5
Matchweek 6
Matchweek 7
Matchweek 8
Matchweek 9
Matchweek 10
Matchweek 11
Matchweek 12
Matchweek 13
Matchweek 14
Matchweek 15
Matchweek 16
Matchweek 17
Matchweek 18
Matchweek 19
Matchweek 20
Matchweek 21
Matchweek 22
Matchweek 23
Matchweek 24
Matchweek 25
Matchweek 26
Matchweek 27
Matchweek 28
Matchweek 29
Matchweek 30
Matchweek 32
Matchweek 33
Matchweek 34
Matchweek 35
Matchweek 31
Matchweek 36
Matchweek 37
Matchweek 38
West-Ham-United-Stats
-------
Matchweek 1
Matchweek 2
Matchweek 3
Matchweek 4
Matchweek 5
Matchweek 6
Matchweek 7
Matchweek 8
Matchweek 9
Matchweek 10
Matchweek 11
Matchweek 12
Matchweek 13
Matchweek 14
Matchweek 15
Matchweek 16
Matchweek 17
Matchweek 18
ERREUR : Matchweek 19
Matchweek 20
Matchweek 21
Matchweek 22
Matchweek 23
Matchweek 24
Matchweek 25
Matchweek 26
Matchweek 27
Matchweek 28
Matchweek 29
Matchweek 30
Matchweek 31
Matchweek 32
Matchweek 33
Matchweek 34
Matchweek 35
Matchwee

In [6]:
dir="./2017-2018/"
exception_pp=[]
equipes=os.listdir(dir)
for equipe in equipes:
    if "Stats" in equipe:
        try:
            statistiques=pd.read_csv(f"{dir}{equipe}/{equipe}.csv")
            print(equipe)
            print("-------")
            for index,round in statistiques.iterrows():
                matchweek=round['Round']
                try:
                    if "History" not in round['Match Report']:
                        performence(round['Match Report'],round['Venue'],f"{dir}{equipe}/{matchweek}")
                        print(matchweek)
                        time.sleep(2)
                except:
                    exception_pp.append([equipe,matchweek,round['Match Report']])
                    print("ERREUR : "+matchweek)
        except:
            pass
header=["team","matchweek","Match Report"]
erreur_pp = pd.DataFrame(exception_pp, columns=header)
erreur_pp.to_csv(f"{dir}/erreur_pp.csv")

Arsenal-Stats
-------


Matchweek 1
Matchweek 2
Matchweek 3
Matchweek 4
Matchweek 5
Matchweek 6
Matchweek 7
Matchweek 8
Matchweek 9
ERREUR : Matchweek 10
ERREUR : Matchweek 11
ERREUR : Matchweek 12
ERREUR : Matchweek 13
ERREUR : Matchweek 14
ERREUR : Matchweek 15
ERREUR : Matchweek 16
ERREUR : Matchweek 17
Matchweek 18
Matchweek 19
Matchweek 20
Matchweek 21
Matchweek 22
Matchweek 23
Matchweek 24
Matchweek 25
Matchweek 26
Matchweek 27
Matchweek 28
Matchweek 29
Matchweek 30
Matchweek 32
Matchweek 33
Matchweek 34
Matchweek 35
Matchweek 36
Matchweek 37
Matchweek 31
Matchweek 38
West-Ham-United-Stats
-------
Matchweek 1
ERREUR : Matchweek 2
Matchweek 3
Matchweek 4
Matchweek 5
Matchweek 6
Matchweek 7
Matchweek 8
Matchweek 9
Matchweek 10
Matchweek 11
Matchweek 12
Matchweek 13
Matchweek 14
Matchweek 15
Matchweek 16
Matchweek 17
Matchweek 18
Matchweek 19
Matchweek 20
Matchweek 22
Matchweek 21
ERREUR : Matchweek 23
Matchweek 24
Matchweek 25
Matchweek 26
Matchweek 27
Matchweek 28
Matchweek 29
Matchweek 30
Matchweek 32
M

In [3]:
dir="./2016-2017/"
exception_pp=[]
equipes=os.listdir(dir)
for equipe in equipes:
    if "Stats" in equipe:
        try:
            statistiques=pd.read_csv(f"{dir}{equipe}/{equipe}.csv")
            print(equipe)
            print("-------")
            for index,round in statistiques.iterrows():
                matchweek=round['Round']
                try:
                    if "History" not in round['Match Report']:
                        performence(round['Match Report'],round['Venue'],f"{dir}{equipe}/{matchweek}")
                        print(matchweek)
                        time.sleep(2)
                except:
                    exception_pp.append([equipe,matchweek,round['Match Report']])
                    print("ERREUR : "+matchweek)
        except:
            pass
header=["team","matchweek","Match Report"]
erreur_pp = pd.DataFrame(exception_pp, columns=header)
erreur_pp.to_csv(f"{dir}/erreur_pp.csv")

Arsenal-Stats
-------
ERREUR : Matchweek 1
ERREUR : Matchweek 2
ERREUR : Matchweek 3
ERREUR : Matchweek 4
ERREUR : Matchweek 5
ERREUR : Matchweek 6
ERREUR : Matchweek 7
ERREUR : Matchweek 8
ERREUR : Matchweek 9
ERREUR : Matchweek 10
ERREUR : Matchweek 11
ERREUR : Matchweek 12
ERREUR : Matchweek 13
ERREUR : Matchweek 14
ERREUR : Matchweek 15
ERREUR : Matchweek 16
ERREUR : Matchweek 17
ERREUR : Matchweek 18
ERREUR : Matchweek 19
ERREUR : Matchweek 20
ERREUR : Matchweek 21
ERREUR : Matchweek 22
ERREUR : Matchweek 23
ERREUR : Matchweek 24
ERREUR : Matchweek 25
ERREUR : Matchweek 27
ERREUR : Matchweek 29
ERREUR : Matchweek 30
ERREUR : Matchweek 31
ERREUR : Matchweek 32
ERREUR : Matchweek 33
ERREUR : Matchweek 28
ERREUR : Matchweek 35
ERREUR : Matchweek 36
ERREUR : Matchweek 26
ERREUR : Matchweek 37
ERREUR : Matchweek 34
ERREUR : Matchweek 38
West-Ham-United-Stats
-------
ERREUR : Matchweek 1
ERREUR : Matchweek 2
ERREUR : Matchweek 3
ERREUR : Matchweek 4
ERREUR : Matchweek 5
ERREUR : Matchwe

In [6]:
dir="./2019-2020/"
exception_pp=[]
equipes=os.listdir(dir)
for equipe in equipes:
    if "Stats" in equipe:
        try:
            statistiques=pd.read_csv(f"{dir}{equipe}/{equipe}.csv")
            print(equipe)
            print("-------")
            for index,round in statistiques.iterrows():
                matchweek=round['Round']
                try:
                    if "History" not in round['Match Report']:
                        performence(round['Match Report'],round['Venue'],f"{dir}{equipe}/{matchweek}")
                        print(matchweek)
                        time.sleep(2)
                except:
                    exception_pp.append([equipe,matchweek,round['Match Report']])
                    print("ERREUR : "+matchweek)
        except:
            pass
header=["team","matchweek","Match Report"]
erreur_pp = pd.DataFrame(exception_pp, columns=header)
erreur_pp.to_csv(f"{dir}/erreur_pp.csv")

Arsenal-Stats
-------
ERREUR : Matchweek 1
ERREUR : Matchweek 2
ERREUR : Matchweek 3
ERREUR : Matchweek 4
ERREUR : Matchweek 5
ERREUR : Matchweek 6
ERREUR : Matchweek 7
ERREUR : Matchweek 8
ERREUR : Matchweek 9
ERREUR : Matchweek 10
ERREUR : Matchweek 11
ERREUR : Matchweek 12
ERREUR : Matchweek 13
ERREUR : Matchweek 14
ERREUR : Matchweek 15
ERREUR : Matchweek 16
ERREUR : Matchweek 17
ERREUR : Matchweek 18
ERREUR : Matchweek 19
ERREUR : Matchweek 20
ERREUR : Matchweek 21
ERREUR : Matchweek 22
ERREUR : Matchweek 23
ERREUR : Matchweek 24
ERREUR : Matchweek 25
ERREUR : Matchweek 26
ERREUR : Matchweek 27
ERREUR : Matchweek 29
ERREUR : Matchweek 28
ERREUR : Matchweek 30
ERREUR : Matchweek 31
ERREUR : Matchweek 32
ERREUR : Matchweek 33
ERREUR : Matchweek 34
ERREUR : Matchweek 35
ERREUR : Matchweek 36
ERREUR : Matchweek 37
ERREUR : Matchweek 38
West-Ham-United-Stats
-------
ERREUR : Matchweek 1
ERREUR : Matchweek 2
ERREUR : Matchweek 3
ERREUR : Matchweek 4
ERREUR : Matchweek 5
ERREUR : Matchwe

## LineUp

In [8]:
def lineUp(url,club,dir):
    req=requests.get(url)
    soup=BeautifulSoup(req.text)
    lineUp = soup.select("div.lineup table tr")
    equipes = [(lineUp[i].select("th"))[0].get_text() for i in [0,len(lineUp)//2]]
    lineUp=[[td.get_text() for td in tr.select("td")] for tr in lineUp]
    header=["num","joueur"]
    if(club not in equipes[0]):
        lineUp=lineUp[1:12][:]
    else:
         lineUp=lineUp[23:34][:]
    df = pd.DataFrame(lineUp, columns=header)
    df.to_csv(f"{dir}/lineup.csv")
    

In [9]:
dir="./2011-2012"
exception=[]
equipes=os.listdir(dir)
for equipe in equipes:
    if "Stats" in equipe:
        statistiques=pd.read_csv(f"{dir}/{equipe}/{equipe}.csv")
        print(equipe)
        print("-------")
        for index,round in statistiques.iterrows():
            matchweek=round['Round']
            try:
                os.mkdir(f"{dir}/{equipe}/{matchweek}")
                lineUp(round['Match Report'],round['Opponent'],f"{dir}/{equipe}/{matchweek}")
                print(matchweek)
            except:
                exception.append([equipe,matchweek,round['Match Report']])
                print("ERREUR : "+matchweek)
header=["team","matchweek","Match Report"]
erreur_lineUp = pd.DataFrame(exception, columns=header)
erreur_lineUp.to_csv(f"{dir}/erreur_lineUp.csv")

Arsenal-Stats
-------
ERREUR : Matchweek 1
ERREUR : Matchweek 2
ERREUR : Matchweek 3
ERREUR : Matchweek 4
ERREUR : Matchweek 5
ERREUR : Matchweek 6
ERREUR : Matchweek 7
ERREUR : Matchweek 8
ERREUR : Matchweek 9
ERREUR : Matchweek 10
ERREUR : Matchweek 11
ERREUR : Matchweek 12
ERREUR : Matchweek 13
ERREUR : Matchweek 14
ERREUR : Matchweek 15
ERREUR : Matchweek 16
ERREUR : Matchweek 17
ERREUR : Matchweek 18
ERREUR : Matchweek 19
ERREUR : Matchweek 20
ERREUR : Matchweek 21
ERREUR : Matchweek 22
ERREUR : Matchweek 23
ERREUR : Matchweek 24
ERREUR : Matchweek 25
ERREUR : Matchweek 26
ERREUR : Matchweek 27
ERREUR : Matchweek 28
ERREUR : Matchweek 29
ERREUR : Matchweek 30
ERREUR : Matchweek 31
ERREUR : Matchweek 32
ERREUR : Matchweek 33
ERREUR : Matchweek 34
ERREUR : Matchweek 35
ERREUR : Matchweek 36
ERREUR : Matchweek 37
ERREUR : Matchweek 38
Newcastle-United-Stats
-------
ERREUR : Matchweek 1
ERREUR : Matchweek 2
ERREUR : Matchweek 3
ERREUR : Matchweek 4
ERREUR : Matchweek 5
ERREUR : Matchw

In [35]:
dir="./2023-2024"
exception=[]
equipes=os.listdir(dir)
for equipe in equipes:
    if equipe != "listeEquipes.csv":
        try:
            statistiques=pd.read_csv(f"{dir}/{equipe}/{equipe}.csv")
            print(equipe)
            print("-------")
            for index,round in statistiques.iterrows():
                matchweek=round['Round']
                try:
                    if "History" not in round['Match Report']:
                        os.mkdir(f"{dir}/{equipe}/{matchweek}")
                        print(matchweek)
                        lineUp(round['Match Report'],round['Opponent'],f"{dir}/{equipe}/{matchweek}")
                except:
                    exception.append([equipe,matchweek,round['Match Report']])
                    print("ERREUR : "+matchweek)
        except:
            pass
header=["team","matchweek","Match Report"]
erreur_lineUp = pd.DataFrame(exception, columns=header)
erreur_lineUp.to_csv(f"{dir}/erreur_lineUp.csv")

# up

In [3]:
dir="./2011-2012/"
exception_pp=[]
equipes=os.listdir(dir)
for equipe in equipes:
    if "Stats" in equipe:
        try:
            statistiques=pd.read_csv(f"{dir}{equipe}/{equipe}.csv")
            print(equipe)
            print("-------")
            for index,round in statistiques.iterrows():
                matchweek=round['Round']
                try:
                    if "History" not in round['Match Report']:
                        performence(round['Match Report'],round['Venue'],f"{dir}{equipe}/{matchweek}")
                        print(matchweek)
                        time.sleep(2)
                except:
                    exception_pp.append([equipe,matchweek,round['Match Report']])
                    print("ERREUR : "+matchweek)
        except:
            pass
header=["team","matchweek","Match Report"]
erreur_pp = pd.DataFrame(exception_pp, columns=header)
erreur_pp.to_csv(f"{dir}/erreur_pp.csv")

Arsenal-Stats
-------
ERREUR : Matchweek 1
ERREUR : Matchweek 2
ERREUR : Matchweek 3
ERREUR : Matchweek 4
ERREUR : Matchweek 5
ERREUR : Matchweek 6
ERREUR : Matchweek 7
ERREUR : Matchweek 8
ERREUR : Matchweek 9
ERREUR : Matchweek 10
ERREUR : Matchweek 11
ERREUR : Matchweek 12


In [ ]:
dir="./2012-2013/"
exception_pp=[]
equipes=os.listdir(dir)
for equipe in equipes:
    if "Stats" in equipe:
        try:
            statistiques=pd.read_csv(f"{dir}{equipe}/{equipe}.csv")
            print(equipe)
            print("-------")
            for index,round in statistiques.iterrows():
                matchweek=round['Round']
                try:
                    if "History" not in round['Match Report']:
                        performence(round['Match Report'],round['Venue'],f"{dir}{equipe}/{matchweek}")
                        print(matchweek)
                        time.sleep(2)
                except:
                    exception_pp.append([equipe,matchweek,round['Match Report']])
                    print("ERREUR : "+matchweek)
        except:
            pass
header=["team","matchweek","Match Report"]
erreur_pp = pd.DataFrame(exception_pp, columns=header)
erreur_pp.to_csv(f"{dir}/erreur_pp.csv")

In [ ]:
dir="./2013-2014/"
exception_pp=[]
equipes=os.listdir(dir)
for equipe in equipes:
    if "Stats" in equipe:
        try:
            statistiques=pd.read_csv(f"{dir}{equipe}/{equipe}.csv")
            print(equipe)
            print("-------")
            for index,round in statistiques.iterrows():
                matchweek=round['Round']
                try:
                    if "History" not in round['Match Report']:
                        performence(round['Match Report'],round['Venue'],f"{dir}{equipe}/{matchweek}")
                        print(matchweek)
                        time.sleep(2)
                except:
                    exception_pp.append([equipe,matchweek,round['Match Report']])
                    print("ERREUR : "+matchweek)
        except:
            pass
header=["team","matchweek","Match Report"]
erreur_pp = pd.DataFrame(exception_pp, columns=header)
erreur_pp.to_csv(f"{dir}/erreur_pp.csv")

In [ ]:
dir="./2014-2015/"
exception_pp=[]
equipes=os.listdir(dir)
for equipe in equipes:
    if "Stats" in equipe:
        try:
            statistiques=pd.read_csv(f"{dir}{equipe}/{equipe}.csv")
            print(equipe)
            print("-------")
            for index,round in statistiques.iterrows():
                matchweek=round['Round']
                try:
                    if "History" not in round['Match Report']:
                        performence(round['Match Report'],round['Venue'],f"{dir}{equipe}/{matchweek}")
                        print(matchweek)
                        time.sleep(2)
                except:
                    exception_pp.append([equipe,matchweek,round['Match Report']])
                    print("ERREUR : "+matchweek)
        except:
            pass
header=["team","matchweek","Match Report"]
erreur_pp = pd.DataFrame(exception_pp, columns=header)
erreur_pp.to_csv(f"{dir}/erreur_pp.csv")